In [1]:
from pyspark.sql import SparkSession

In [15]:
from pyspark.sql import functions as F

In [21]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

In [2]:
spark = SparkSession.builder \
.appName("Read Data From File") \
.master("local[2]") \
.getOrCreate()

2023-02-22 21:40:25,845 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
! wget -P /home/train/datasets/ \
https://github.com/erkansirin78/datasets/raw/master/dirty_store_transactions.csv 

--2023-02-22 21:44:45--  https://github.com/erkansirin78/datasets/raw/master/dirty_store_transactions.csv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/erkansirin78/datasets/master/dirty_store_transactions.csv [following]
--2023-02-22 21:44:46--  https://raw.githubusercontent.com/erkansirin78/datasets/master/dirty_store_transactions.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2609524 (2.5M) [text/plain]
Saving to: ‘/home/train/datasets/dirty_store_transactions.csv’

100%[======================================>] 2,609,524   2.41MB/s   in 1.0s   

2023-02-22 21:44:48 (2.41 MB/s) - ‘/home

In [7]:
df = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.option("inferSchema", True) \
.load("file:///home/train/datasets/dirty_store_transactions.csv")

In [8]:
df.show()

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|    CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|  YR7220|     New York(|     Electronics|  12254943|$31|$20.77|   $1.86|$29.14|2019-11-26|
|  YR7220|     New York+|       Furniture| 72619323C|$15| $9.75|    $1.5| $13.5|2019-11-26|
|  YR7220|     New York |     Electronics| 34161682B|$88|$62.48|    $4.4| $83.6|2019-11-26|
|  YR7220|     New York!|         Kitchen|  79411621|$91|$58.24|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T|$85|   $51|   $2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204|$37|$24.05|   $0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D|$80| $48.8|    $6.4| $73.6|2019-11-26|
|  YR7220|     New York$|         Kitchen| 52503356^|$71| $42.6|   $5.68|$65.32|

In [10]:
df_pd = df.limit(5).toPandas()

In [12]:
df_pd.head(5)

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date
0,YR7220,New York(,Electronics,12254943,$31,$20.77,$1.86,$29.14,2019-11-26
1,YR7220,New York+,Furniture,72619323C,$15,$9.75,$1.5,$13.5,2019-11-26
2,YR7220,New York,Electronics,34161682B,$88,$62.48,$4.4,$83.6,2019-11-26
3,YR7220,New York!,Kitchen,79411621,$91,$58.24,$3.64,$87.36,2019-11-26
4,YR7220,New York,Fashion,39520263T,$85,$51,$2.55,$82.45,2019-11-26


# 1 STORE_LOCATION

In [65]:
df.select('STORE_LOCATION').distinct().collect()

[Row(STORE_LOCATION="Miami'"),
 Row(STORE_LOCATION='New York""'),
 Row(STORE_LOCATION='Washington""'),
 Row(STORE_LOCATION='Miami)'),
 Row(STORE_LOCATION='Houston%'),
 Row(STORE_LOCATION='Miami&'),
 Row(STORE_LOCATION='Houston$'),
 Row(STORE_LOCATION='New York$'),
 Row(STORE_LOCATION="New York'"),
 Row(STORE_LOCATION='New York+'),
 Row(STORE_LOCATION='New York('),
 Row(STORE_LOCATION='New York!'),
 Row(STORE_LOCATION='Miami#'),
 Row(STORE_LOCATION='Houston*'),
 Row(STORE_LOCATION='New York '),
 Row(STORE_LOCATION='New York&'),
 Row(STORE_LOCATION='Miami$'),
 Row(STORE_LOCATION='Denver$'),
 Row(STORE_LOCATION="Houston'"),
 Row(STORE_LOCATION='Denver+'),
 Row(STORE_LOCATION='Houston('),
 Row(STORE_LOCATION='Washington'),
 Row(STORE_LOCATION='New York)'),
 Row(STORE_LOCATION='Houston)'),
 Row(STORE_LOCATION="Denver'"),
 Row(STORE_LOCATION='Miami%'),
 Row(STORE_LOCATION='Miami '),
 Row(STORE_LOCATION='Washington$'),
 Row(STORE_LOCATION='Denver*'),
 Row(STORE_LOCATION='Miami+'),
 Row(STORE_

In [66]:
df1= df.withColumn("STORE_LOCATION", F.translate("STORE_LOCATION", "+!$'""%&*#+(", "          "))
df1.show()

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|    CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|  YR7220|     New York |     Electronics|  12254943|$31|$20.77|   $1.86|$29.14|2019-11-26|
|  YR7220|     New York |       Furniture| 72619323C|$15| $9.75|    $1.5| $13.5|2019-11-26|
|  YR7220|     New York |     Electronics| 34161682B|$88|$62.48|    $4.4| $83.6|2019-11-26|
|  YR7220|     New York |         Kitchen|  79411621|$91|$58.24|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T|$85|   $51|   $2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204|$37|$24.05|   $0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D|$80| $48.8|    $6.4| $73.6|2019-11-26|
|  YR7220|     New York |         Kitchen| 52503356^|$71| $42.6|   $5.68|$65.32|

In [67]:
df1.select('STORE_LOCATION').distinct().collect()

[Row(STORE_LOCATION='New York""'),
 Row(STORE_LOCATION='Washington""'),
 Row(STORE_LOCATION='Miami)'),
 Row(STORE_LOCATION='New York '),
 Row(STORE_LOCATION='Washington'),
 Row(STORE_LOCATION='New York)'),
 Row(STORE_LOCATION='Houston)'),
 Row(STORE_LOCATION='Miami '),
 Row(STORE_LOCATION='Houston""'),
 Row(STORE_LOCATION='Washington '),
 Row(STORE_LOCATION='Miami""'),
 Row(STORE_LOCATION='Washington)'),
 Row(STORE_LOCATION='Houston'),
 Row(STORE_LOCATION='Denver)'),
 Row(STORE_LOCATION='Houston '),
 Row(STORE_LOCATION='Denver""'),
 Row(STORE_LOCATION='Miami'),
 Row(STORE_LOCATION='New York'),
 Row(STORE_LOCATION='Denver '),
 Row(STORE_LOCATION='Denver')]

In [68]:
df2= df1.withColumn("STORE_LOCATION", F.translate("STORE_LOCATION", '"', " "))

In [69]:
df2.select('STORE_LOCATION').distinct().collect()

[Row(STORE_LOCATION='Miami)'),
 Row(STORE_LOCATION='Denver  '),
 Row(STORE_LOCATION='New York '),
 Row(STORE_LOCATION='Washington'),
 Row(STORE_LOCATION='New York)'),
 Row(STORE_LOCATION='Houston)'),
 Row(STORE_LOCATION='Miami '),
 Row(STORE_LOCATION='Washington '),
 Row(STORE_LOCATION='Miami  '),
 Row(STORE_LOCATION='Washington)'),
 Row(STORE_LOCATION='Houston'),
 Row(STORE_LOCATION='Denver)'),
 Row(STORE_LOCATION='New York  '),
 Row(STORE_LOCATION='Houston '),
 Row(STORE_LOCATION='Houston  '),
 Row(STORE_LOCATION='Washington  '),
 Row(STORE_LOCATION='Miami'),
 Row(STORE_LOCATION='New York'),
 Row(STORE_LOCATION='Denver '),
 Row(STORE_LOCATION='Denver')]

In [71]:
df3= df2.withColumn("STORE_LOCATION", F.translate("STORE_LOCATION", ')', " "))

In [72]:
df3.select('STORE_LOCATION').distinct().collect()

[Row(STORE_LOCATION='Denver  '),
 Row(STORE_LOCATION='New York '),
 Row(STORE_LOCATION='Washington'),
 Row(STORE_LOCATION='Miami '),
 Row(STORE_LOCATION='Washington '),
 Row(STORE_LOCATION='Miami  '),
 Row(STORE_LOCATION='Houston'),
 Row(STORE_LOCATION='New York  '),
 Row(STORE_LOCATION='Houston '),
 Row(STORE_LOCATION='Houston  '),
 Row(STORE_LOCATION='Washington  '),
 Row(STORE_LOCATION='Miami'),
 Row(STORE_LOCATION='New York'),
 Row(STORE_LOCATION='Denver '),
 Row(STORE_LOCATION='Denver')]

In [73]:
df3.select(F.trim("STORE_LOCATION").alias("STORE_LOCATION")).show()

+--------------+
|STORE_LOCATION|
+--------------+
|      New York|
|      New York|
|      New York|
|      New York|
|      New York|
|      New York|
|      New York|
|      New York|
|      New York|
|      New York|
|      New York|
|      New York|
|      New York|
|      New York|
|      New York|
|      New York|
|      New York|
|      New York|
|      New York|
|      New York|
+--------------+
only showing top 20 rows



In [77]:
df3.withColumn("STORE_LOCATION", F.trim("STORE_LOCATION").alias("STORE_LOCATION")).show()

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|    CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|  YR7220|      New York|     Electronics|  12254943|$31|$20.77|   $1.86|$29.14|2019-11-26|
|  YR7220|      New York|       Furniture| 72619323C|$15| $9.75|    $1.5| $13.5|2019-11-26|
|  YR7220|      New York|     Electronics| 34161682B|$88|$62.48|    $4.4| $83.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  79411621|$91|$58.24|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T|$85|   $51|   $2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204|$37|$24.05|   $0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D|$80| $48.8|    $6.4| $73.6|2019-11-26|
|  YR7220|      New York|         Kitchen| 52503356^|$71| $42.6|   $5.68|$65.32|

In [78]:
df4 = df3.withColumn("STORE_LOCATION", F.trim("STORE_LOCATION").alias("STORE_LOCATION"))

In [79]:
df4.select('STORE_LOCATION').distinct().collect()

[Row(STORE_LOCATION='Washington'),
 Row(STORE_LOCATION='Houston'),
 Row(STORE_LOCATION='Miami'),
 Row(STORE_LOCATION='New York'),
 Row(STORE_LOCATION='Denver')]

# 2

In [80]:
df4.show()

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|    CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|  YR7220|      New York|     Electronics|  12254943|$31|$20.77|   $1.86|$29.14|2019-11-26|
|  YR7220|      New York|       Furniture| 72619323C|$15| $9.75|    $1.5| $13.5|2019-11-26|
|  YR7220|      New York|     Electronics| 34161682B|$88|$62.48|    $4.4| $83.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  79411621|$91|$58.24|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T|$85|   $51|   $2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204|$37|$24.05|   $0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D|$80| $48.8|    $6.4| $73.6|2019-11-26|
|  YR7220|      New York|         Kitchen| 52503356^|$71| $42.6|   $5.68|$65.32|

In [81]:
df4.select('PRODUCT_CATEGORY').distinct().collect()

[Row(PRODUCT_CATEGORY='Education'),
 Row(PRODUCT_CATEGORY='Kitchen'),
 Row(PRODUCT_CATEGORY='Fashion'),
 Row(PRODUCT_CATEGORY='Groceries'),
 Row(PRODUCT_CATEGORY='Electronics'),
 Row(PRODUCT_CATEGORY='Furniture'),
 Row(PRODUCT_CATEGORY='Cosmetics')]

# 3

In [82]:
df4.select('MRP').distinct().collect()

[Row(MRP='$93'),
 Row(MRP='$92'),
 Row(MRP='$68'),
 Row(MRP='$11'),
 Row(MRP='$26'),
 Row(MRP='$86'),
 Row(MRP='$75'),
 Row(MRP='$95'),
 Row(MRP='$73'),
 Row(MRP='$16'),
 Row(MRP='$83'),
 Row(MRP='$13'),
 Row(MRP='$66'),
 Row(MRP='$67'),
 Row(MRP='$56'),
 Row(MRP='$42'),
 Row(MRP='$79'),
 Row(MRP='$70'),
 Row(MRP='$69'),
 Row(MRP='$74'),
 Row(MRP='$57'),
 Row(MRP='$41'),
 Row(MRP='$34'),
 Row(MRP='$35'),
 Row(MRP='$77'),
 Row(MRP='$37'),
 Row(MRP='$89'),
 Row(MRP='$20'),
 Row(MRP='$96'),
 Row(MRP='$88'),
 Row(MRP='$23'),
 Row(MRP='$39'),
 Row(MRP='$12'),
 Row(MRP='$62'),
 Row(MRP='$46'),
 Row(MRP='$50'),
 Row(MRP='$44'),
 Row(MRP='$36'),
 Row(MRP='$25'),
 Row(MRP='$87'),
 Row(MRP='$97'),
 Row(MRP='$19'),
 Row(MRP='$29'),
 Row(MRP='$33'),
 Row(MRP='$71'),
 Row(MRP='$21'),
 Row(MRP='$81'),
 Row(MRP='$31'),
 Row(MRP='$55'),
 Row(MRP='$100'),
 Row(MRP='$80'),
 Row(MRP='$78'),
 Row(MRP='$98'),
 Row(MRP='$38'),
 Row(MRP='$28'),
 Row(MRP='$17'),
 Row(MRP='$84'),
 Row(MRP='$59'),
 Row(MRP='$72

In [83]:
df5= df4.withColumn("MRP", F.translate("MRP", '$', ""))

In [84]:
df5.select('MRP').distinct().collect()

[Row(MRP='51'),
 Row(MRP='15'),
 Row(MRP='54'),
 Row(MRP='11'),
 Row(MRP='69'),
 Row(MRP='29'),
 Row(MRP='42'),
 Row(MRP='73'),
 Row(MRP='87'),
 Row(MRP='64'),
 Row(MRP='30'),
 Row(MRP='34'),
 Row(MRP='59'),
 Row(MRP='28'),
 Row(MRP='22'),
 Row(MRP='85'),
 Row(MRP='16'),
 Row(MRP='52'),
 Row(MRP='35'),
 Row(MRP='71'),
 Row(MRP='98'),
 Row(MRP='99'),
 Row(MRP='47'),
 Row(MRP='96'),
 Row(MRP='43'),
 Row(MRP='31'),
 Row(MRP='100'),
 Row(MRP='18'),
 Row(MRP='70'),
 Row(MRP='27'),
 Row(MRP='61'),
 Row(MRP='75'),
 Row(MRP='17'),
 Row(MRP='26'),
 Row(MRP='46'),
 Row(MRP='78'),
 Row(MRP='77'),
 Row(MRP='89'),
 Row(MRP='60'),
 Row(MRP='68'),
 Row(MRP='90'),
 Row(MRP='19'),
 Row(MRP='41'),
 Row(MRP='23'),
 Row(MRP='55'),
 Row(MRP='93'),
 Row(MRP='95'),
 Row(MRP='38'),
 Row(MRP='40'),
 Row(MRP='25'),
 Row(MRP='44'),
 Row(MRP='82'),
 Row(MRP='53'),
 Row(MRP='92'),
 Row(MRP='86'),
 Row(MRP='58'),
 Row(MRP='81'),
 Row(MRP='33'),
 Row(MRP='48'),
 Row(MRP='97'),
 Row(MRP='67'),
 Row(MRP='84'),
 Row(MR

# 4

In [85]:
df5.show()

+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|    CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|  YR7220|      New York|     Electronics|  12254943| 31|$20.77|   $1.86|$29.14|2019-11-26|
|  YR7220|      New York|       Furniture| 72619323C| 15| $9.75|    $1.5| $13.5|2019-11-26|
|  YR7220|      New York|     Electronics| 34161682B| 88|$62.48|    $4.4| $83.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  79411621| 91|$58.24|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T| 85|   $51|   $2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204| 37|$24.05|   $0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D| 80| $48.8|    $6.4| $73.6|2019-11-26|
|  YR7220|      New York|         Kitchen| 52503356^| 71| $42.6|   $5.68|$65.32|

In [88]:
df6= df5.withColumn("CP", F.translate("CP", '$', ""))

In [89]:
df6.show()

+--------+--------------+----------------+----------+---+-----+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|   CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+-----+--------+------+----------+
|  YR7220|      New York|     Electronics|  12254943| 31|20.77|   $1.86|$29.14|2019-11-26|
|  YR7220|      New York|       Furniture| 72619323C| 15| 9.75|    $1.5| $13.5|2019-11-26|
|  YR7220|      New York|     Electronics| 34161682B| 88|62.48|    $4.4| $83.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  79411621| 91|58.24|   $3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T| 85|   51|   $2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204| 37|24.05|   $0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D| 80| 48.8|    $6.4| $73.6|2019-11-26|
|  YR7220|      New York|         Kitchen| 52503356^| 71| 42.6|   $5.68|$65.32|2019-11-26|

In [90]:
df7= df6.withColumn("DISCOUNT", F.translate("DISCOUNT", '$', ""))

In [91]:
df7.show()

+--------+--------------+----------------+----------+---+-----+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|   CP|DISCOUNT|    SP|      Date|
+--------+--------------+----------------+----------+---+-----+--------+------+----------+
|  YR7220|      New York|     Electronics|  12254943| 31|20.77|    1.86|$29.14|2019-11-26|
|  YR7220|      New York|       Furniture| 72619323C| 15| 9.75|     1.5| $13.5|2019-11-26|
|  YR7220|      New York|     Electronics| 34161682B| 88|62.48|     4.4| $83.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  79411621| 91|58.24|    3.64|$87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T| 85|   51|    2.55|$82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204| 37|24.05|    0.74|$36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D| 80| 48.8|     6.4| $73.6|2019-11-26|
|  YR7220|      New York|         Kitchen| 52503356^| 71| 42.6|    5.68|$65.32|2019-11-26|

In [92]:
df8= df7.withColumn("SP", F.translate("SP", '$', ""))

In [93]:
df8.show()

+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|   CP|DISCOUNT|   SP|      Date|
+--------+--------------+----------------+----------+---+-----+--------+-----+----------+
|  YR7220|      New York|     Electronics|  12254943| 31|20.77|    1.86|29.14|2019-11-26|
|  YR7220|      New York|       Furniture| 72619323C| 15| 9.75|     1.5| 13.5|2019-11-26|
|  YR7220|      New York|     Electronics| 34161682B| 88|62.48|     4.4| 83.6|2019-11-26|
|  YR7220|      New York|         Kitchen|  79411621| 91|58.24|    3.64|87.36|2019-11-26|
|  YR7220|      New York|         Fashion| 39520263T| 85|   51|    2.55|82.45|2019-11-26|
|  YR7220|      New York|         Kitchen|  93809204| 37|24.05|    0.74|36.26|2019-11-26|
|  YR7220|      New York|       Cosmetics| 86610412D| 80| 48.8|     6.4| 73.6|2019-11-26|
|  YR7220|      New York|         Kitchen| 52503356^| 71| 42.6|    5.68|65.32|2019-11-26|
|  YR7220|

# register

In [94]:
! ls -l ~/pyspark_output_data/

total 0
drwxrwxr-x. 2 train train 198 Nov 20  2020 most_cancelled_cats_parquet
drwxrwxr-x. 2 train train 198 Nov 20  2020 most_cancelled_products_parquet


In [98]:
df8.write \
.format("csv") \
.mode("overwrite") \
.save("file:////home/train/datasets/dirty_store_transaction_clear")

In [99]:
! ls -l ~/home/train/venvspark/dev/output_data/

ls: cannot access /home/train/home/train/venvspark/dev/output_data/: No such file or directory


In [100]:
df8.write.csv("file:////home/train/datasets/dirty_store_transaction_clear2.csv", header=True)